<a href="https://colab.research.google.com/github/wxbaum/twitter_influencers_analysis/blob/master/twitter_infuencers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tweepy
import time
import pandas as pd
import json
import time

In [0]:
"""
These keys/tokens require a Twitter developer account. Sign up for a developer 
account at https://developer.twitter.com and create a new app.
""" 

api_key = ''
secret_key = ''
access_token = ''
secret_token = ''

def authenticate(api_key, secret_key, access_token, secret_token):

    auth = tweepy.OAuthHandler(api_key, secret_key)
    auth.set_access_token(access_token, secret_token)
    api = tweepy.API(auth)
  
    return api


api = authenticate(api_key, secret_key, access_token, secret_token)

In [0]:
# Some api calls will result in a rate limit. This functions handles those if needed 
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(5)

To kickstart scraping of influencer accounts, we need a starting point. Here's a top 10 list, and we'll put the account names into a list:
https://www.brandwatch.com/blog/influential-lifestyle-bloggers/


In [0]:
handles_list = ['StacieinAtlanta', 
                'momsreview4you',
                'dawnchats',
                'GiveawayBandit',
                'daddysweeney',
                'pennywisemama',
                'dodomesticdad',
                'TheTypical_Mom',
                'Crissy',
                'EvrydayShortcut']

def get_initial_influencers(handles):
    influencer_id_list = [api.get_user(handle).id for handle in handles_list]
  
    return influencer_id_list
 
  
    initial_influ_ids = get_initial_influencers(handles_list)

We need to look for mentioned accounts that *aren't* brands. If an influencer mentioned or 
retweeted an account that doesn't look like a brand, there's a very good chance that that account is also an influencer promoting themselves or promoting their own product through other influencer channels. We'll take those accounts as influencers. 

In [0]:
def scrape_influencers_use_google(list_contains_string):
    from googlesearch import search
    from random import randint as ranint

    list_urls = []

    urls_checked = 0
    urls_appended = 0
    for url in search("site:twitter.com " + list_contains_string, 
                      start=urls_checked, 
                      num=10, 
                      pause=ranint(3, 5)):
        
        if '/lists/' in url:
            list_urls.append(url)
            urls_appended += 1
            urls_checked += 1

        if urls_checked / urls_appended > 5:
            break
            
        elif urls_appended >= 50:
            break

        time.sleep(ranint(3, 5))

    list_urls = [url[8:] for url in list_urls] # Removes leading https://

    return list_urls
  

list_urls = scrape_influencers_use_google("lists influencers")

In [7]:
print(list_urls)

['twitter.com/ppact/lists/feminist-influencers-list', 'twitter.com/kia/lists/celebrities-influencers', 'twitter.com/jeffbullas/lists/influencers', 'twitter.com/kimgarst/lists/influencers-on-twitter', 'twitter.com/vendhq/lists/top-50-retail-influencers', 'twitter.com/dgingiss/lists/top-25-cx-influencers', 'twitter.com/ausunlimited/lists/top-digital-influencers', 'twitter.com/marketo/lists/influencers-to-follow', 'twitter.com/ideo/lists/design-influencers', 'twitter.com/digitalmumshq/lists/sme-influencers1', 'twitter.com/mipimworld/lists/retail-top-influencers', 'twitter.com/mylessuer/lists/cio-influencers', 'twitter.com/david_perell/lists/tech-influencers', 'twitter.com/milkroundabout/lists/top-tech-influencers', 'twitter.com/guff_se/lists/lean-startup-influencers', 'twitter.com/microsoft/lists/envision-influencers', 'twitter.com/BMCSoftware/lists/cloud-influencers', 'twitter.com/vendhq/lists/top-100-influencers-2018', 'twitter.com/PwC_UK/lists/influencers', 'twitter.com/american_heart/

In [0]:
# Get user id, screen name, bio, follower count

def get_users_in_lists(influencer_lists):
  
    users_list = []
    bios_list = []
    desc_list = []
    follower_count_list = []
    
    for tw_list in influencer_lists:
    
        user = tw_list.split('/')[1]
        list_name = tw_list.split('/')[3]

        list_members_output = api.list_members(user, list_name)

        for user_id in list_members_output:
            users_list.append(user_id._json['id'])
            bios_list.append(user_id._json['screen_name'])
            desc_list.append(user_id._json['description'])
            follower_count_list.append(user_id._json['followers_count'])
            
    zipped_data = zip(users_list, bios_list, desc_list, follower_count_list)
    output_df = pd.DataFrame(list(zipped_data), columns=['user_id', 
                                                         'screen_name',
                                                         'bio',
                                                         'followers'])

    return output_df
  

influencers_df = get_users_in_lists(list_urls)

In [25]:
influencers_df.head()

,user_id,screen_name,bio,followers
0,411194625,RBraceySherman,Black Feminist • 👩🏽‍💻 @AbortionFunds @Abortion...,16676
1,272191896,SarahEMcBride,"Author of Tomorrow Will Be Different, Nat'l Pr...",46901
2,271725689,iamsambee,Alleged humorist.,515025
3,267915933,feministabulous,host of vox media’s consider it | pre-order my...,136544
4,260959284,KathaPollitt,"Politics, poetry, feminism, books, life. New ...",20114
